# Heat transfer with alternative 'difference-star'


In [196]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time

## Parameter Definition and preparation of data structure and initial values

In [197]:
#Size of a discrete 2D-area in grids
N = 25

#Create 2D Matrix for conventional and alternative star
X_conv_start = np.zeros([N,N])
X_conv = np.copy(X_conv_start)

X_alt_Start = np.zeros([N,N])
X_alt = np.copy(X_alt_Start)

## Dirichlet boundary conditions

In [198]:
#value of the field at the lower boundary
X_conv[0:N,0] = 1
X_alt[0:N,0] = 1

#X_conv[0:N,N-1] = 1
#X_alt[0:N,N-1] = 1

## Computation  
Conventional: $T_{i,j}=\frac{1}{4}(T_{i+1,j}+T_{i-1,j}+T_{i,j+1}+T_{i,j-1})$
Neumann BC along left lateral: $T_{0,j}=\frac{1}{4}(T_{0,j+1}+T_{0,j-1}+2\cdot T_{1,j})$
<br>
Alternative: $T_{i,j}=\frac{1}{4}(T_{i+1,j+1}+T_{i+1,j-1}+T_{i-1,j+1}+T_{i-1,j-1})$
Neumann BC along left lateral: $T_{0,j}=\frac{1}{4}(2\cdot T_{1,j+1}+2\cdot T_{1,j-1})$

In [199]:
count_conv = 0
count_alt = 0
time_conv_start = time.perf_counter_ns()
while np.linalg.norm(X_conv-X_conv_start) > 1e-5:
    count_conv= count_conv+1
    X_conv_start = np.copy(X_conv)
    X_conv[1:N-1,1:N-1] = 1/4*(X_conv_start[1:N-1,2:N] + X_conv_start[1:N-1,0:N-2] + X_conv_start[2:N,1:N-1] + X_conv_start[0:N-2,1:N-1])
    X_conv[0,1:N-1] = (X_conv[0,2:N]+X_conv[0,0:N-2]+2*X_conv[1,1:N-1])/4               # Neumann BC along the left lateral
time_conv_elapsed = (time.perf_counter_ns() - time_conv_start)

print("Computation steps with the conventional difference cross: %i" % count_conv)
print("Needed time for them in nanoseconds: %i" % time_conv_elapsed)

time_alt_start = time.perf_counter_ns()
while np.linalg.norm(X_alt-X_alt_Start) > 1e-5:
    count_alt = count_alt + 1
    X_alt_Start = np.copy(X_alt)
    X_alt[1:N-1,1:N-1] = 1/4*(X_alt_Start[2:N,2:N]+X_alt_Start[2:N,0:N-2]+X_alt_Start[0:N-2,2:N]+X_alt_Start[0:N-2,0:N-2])
    X_alt[0,1:N-1] = (2*X_alt[1,2:N]+2*X_alt[1,0:N-2])/4
time_alt_elapsed = (time.perf_counter_ns() - time_conv_start)

print("Computation steps with the alternative difference cross: %i" % count_alt)
print("Needed time for them in nanoseconds: %i" % time_alt_elapsed)
print("Percentage of how many steps the alternative method needs in relation to the conventional one: %.2f%%" % (count_alt/count_conv*100))                     #alternative method needs only half as many steps
print("Percentage of how many nanoseconds the alternative method needs in relation to the conventional one: %.2f%%" % (time_alt_elapsed/time_conv_elapsed*100)) #alternative method is 50% slower

Computation steps with the conventional difference cross: 1531
Needed time for them in nanoseconds: 46396500
Computation steps with the alternative difference cross: 815
Needed time for them in nanoseconds: 73920700
Percentage of how many steps the alternative method needs in relation to the conventional one: 53.23%
Percentage of how many nanoseconds the alternative method needs in relation to the conventional one: 159.32%


## Comparing the Heattransfer solutions

In [200]:
X_diff = X_alt-X_conv

## Visualisation of the results

In [201]:
#coordinates
x = range(N)
y = range(N)
x,y = np.meshgrid(x,y)

#Use this when running in a proper Jupter Notebook environment for interactive plots
#%matplotbib notebook

#Use this when running in VSCode as interactive plot
#requires to install ipympl (pip install ipympl)
#see https://github.com/matplotlib/ipympl for more information
%matplotlib widget

#Plotting heattransfer with conventional difference cross
fig = plt.figure()
ax = fig.add_subplot(131, projection = '3d')
plt.title("Conventional difference star")
ax.plot_surface(x, y, X_conv, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
#Plotting heattransfer with alternative difference cross
ax = fig.add_subplot(132, projection = '3d')
plt.title("Alternative difference star")
ax.plot_surface(x, y, X_alt, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
#Plotting the difference between the two
ax = fig.add_subplot(133, projection = '3d')
plt.title("Difference")
ax.plot_surface(x, y, X_diff, rstride=1, cstride=1, cmap='viridis', edgecolor='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …